<a href="https://colab.research.google.com/github/madhuselvaraj/tensorflow_projects/blob/master/happy_sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing tensorflow and zip file .
Defining callback function inorder to stop the training if it reach 99.9%acuuracy.

In [26]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached 0.999 acc so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()

--2019-04-21 09:53:53--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-04-21 09:53:54 (102 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



Retrieving the path.

In [0]:
train_happy_dir = os.path.join('/tmp/h-or-s/happy')
train_sad_dir = os.path.join('/tmp/h-or-s/sad')


Obtaining the list and total number of images.

In [28]:
train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])
print('total training happy images:', len(os.listdir(train_happy_dir)))
print('total training sad images:', len(os.listdir(train_sad_dir)))

['happy2-06.png', 'happy2-13.png', 'happy1-01.png', 'happy1-02.png', 'happy2-02.png', 'happy1-00.png', 'happy1-19.png', 'happy1-14.png', 'happy2-10.png', 'happy2-03.png']
['sad2-17.png', 'sad2-16.png', 'sad2-11.png', 'sad1-13.png', 'sad2-01.png', 'sad1-00.png', 'sad1-04.png', 'sad1-18.png', 'sad2-03.png', 'sad2-02.png']
total training happy images: 40
total training sad images: 40


designing the model with 3 convolutional layers and loss as 'binary crossentropy'  
sigmoid as activation function
RMS as optimizer

In [0]:
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 9248)              0         
__________

dedine a ImageDataGenerator called train_datagen and a train_generator by calling train_datagen.flow_from_directory.

In [36]:
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',  
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 80 images belonging to 2 classes.


In [37]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2,  
      epochs=15,
      verbose=1,
callbacks=[callbacks])

Epoch 1/15
8/8 [==============================] - 3s 326ms/step - loss: 1.0111 - acc: 0.5750
Epoch 2/15
8/8 [==============================] - 2s 306ms/step - loss: 0.5381 - acc: 0.8000
Epoch 3/15
8/8 [==============================] - 2s 297ms/step - loss: 0.2761 - acc: 0.8625
Epoch 4/15
8/8 [==============================] - 2s 305ms/step - loss: 0.2374 - acc: 0.9000
Epoch 5/15
8/8 [==============================] - 2s 296ms/step - loss: 0.2191 - acc: 0.8750
Epoch 6/15
8/8 [==============================] - 2s 294ms/step - loss: 0.1329 - acc: 0.9375
Epoch 7/15
8/8 [==============================] - 2s 300ms/step - loss: 0.1065 - acc: 0.9625
Epoch 8/15
8/8 [==============================] - 2s 301ms/step - loss: 0.0498 - acc: 0.9875
Epoch 9/15
8/8 [==============================] - 2s 289ms/step - loss: 0.0334 - acc: 0.9875
Epoch 10/15
8/8 [==============================] - 2s 304ms/step - loss: 0.0691 - acc: 0.9625
Epoch 11/15
8/8 [==============================] - 2s 298ms/step - lo

fitting the model.

<--------------------------------------------validation---------------------------------------->

validating the model through uploading a validation image for prediction.

In [38]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=2)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a happy")
  else:
    print(fn + " is a sad")

Saving person-731165_960_720.jpg to person-731165_960_720.jpg
[0.]
person-731165_960_720.jpg is a sad
